#Create a Q&A bot

Today we will be building a simple conversational chatbot with an interesting generative model. And we will be deploying it with gradio and hugging face spaces: facebook/blenderbot-400M-distill is a lightweight, distilled version of Facebook AI's BlenderBot model, designed for open-domain conversational AI. It has 400 million parameters and is optimized for efficiency while maintaining good performance in generating human-like, context-aware responses. The model is ideal for chatbots and dialogue systems, offering capabilities for casual conversation, answering questions, and engaging in meaningful interactions. It is available via the Hugging Face Transformers library for easy integration into applications.

**Note:** Please note that the model used in this project is a basic, lightweight version, not intended for handling complex queries. For more advanced and robust LLMs, you can explore a wide range of options at huggingface.com.

##Installing necessary libraries

In [1]:
!pip install gradio transformers huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.4/321.4 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 122.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 7.0 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


##First implementation with Gradio interface (gr.Interface())

In [7]:
import torch
import gradio as gr
from transformers import LlamaForCausalLM, LlamaTokenizer
from huggingface_hub import login

# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import AutoTokenizer, AutoModelForCausalLM

# Replace with your Hugging Face token
HF_ACCESS_TOKEN = "Hugging Face token"

# Authenticate
login(token=HF_ACCESS_TOKEN)

# Specify the model name
model_name = "facebook/blenderbot-400M-distill"


# Load the tokenizer and model locally
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Generate a response
def generate_response(prompt_txt):
    try:
        # Encode the input text
        inputs = tokenizer(prompt_txt, return_tensors="pt")

        # Generate a response using the model
        outputs = model.generate(**inputs, max_new_tokens=250, temperature=0.5)

        # Decode the generated text
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        return generated_text
    except Exception as e:
        return f"Error generating response: {e}"

# Example usage
#prompt = "What is the capital of France?"
#response = generate_response(prompt)
#print(response)

# Create Gradio interface
chat_application = gr.Interface(
    fn=generate_response,
    allow_flagging="never",
    inputs=gr.Textbox(label="Input", lines=2, placeholder="Type your question here..."),
    outputs=gr.Textbox(label="Output"),
    title="blenderbot-400M-distill",
    description="Ask any question and the chatbot will try to answer."
)

# Launch Gradio app
chat_application.launch(share=True)


/usr/local/lib/python3.10/dist-packages/gradio/interface.py:403: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://60ff6c327973c0c125.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [3]:
# Example usage
prompt = "What is the capital of France?"
response = generate_response(prompt)
print(response)

 The capital is Paris. It is the most populous city in the French Republic.


##Second implementation with Gradio Block (gr.Block())

In [4]:
# Import required libraries
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import gradio as gr

# Choosing a model
model_name = "facebook/blenderbot-400M-distill"

# Fetch the model and initialize a tokenizer
# Load model (download on first run and reference local installation for consequent runs)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Keeping track of conversation history
conversation_history = []

# Function to generate a chatbot response
def chatbot_response(user_input):
    global conversation_history

    # Create conversation history string
    history_string = "\n".join(conversation_history)

    # Tokenize the input text and history
    inputs = tokenizer.encode_plus(history_string, user_input, return_tensors="pt")

    # Generate the response from the model
    outputs = model.generate(**inputs)

    # Decode the response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    # Add interaction to conversation history
    conversation_history.append(user_input)
    conversation_history.append(response)

    return response

# Function to reset the conversation history
def reset_conversation():
    global conversation_history
    conversation_history = []
    return "Conversation reset. How can I assist you?"


# Create Gradio interface using with gr.Blocks() to manage the context:
with gr.Blocks() as interface:  # Create a Blocks context
    # Define input and output components
    input_box = gr.Textbox(label="Your Message", placeholder="Type your question here...")
    output_box = gr.Textbox(label="Bot Response")

    # Create a submit button
    submit_button = gr.Button("Submit")

    # Create a reset button
    reset_button = gr.Button("Reset Conversation")

    # Link the submit button to the chatbot_response function
    submit_button.click(fn=chatbot_response, inputs=input_box, outputs=output_box)

    # Link the reset button to the reset function
    reset_button.click(fn=reset_conversation, outputs=output_box)

# Set title and description
interface.title = "BlenderBot Chatbot"
interface.description = "A conversational AI chatbot powered by BlenderBot. Ask anything!"
interface.examples = [
    ["Hello!"],
    ["Can you tell me a joke?"],
    ["What is the capital of France?"],
]
interface.live = True

# Launch the Gradio app (for local testing)
if __name__ == "__main__":
    interface.launch()

# For deployment on Hugging Face Spaces, save this script as `app.py` and push to your Hugging Face Space repository.

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://863b52b0ae2b85c755.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


##Third implementation with Gradio Block (gr.Block()) and integrating chat history

In [5]:
# Import required libraries
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import gradio as gr

# Choosing a model
model_name = "facebook/blenderbot-400M-distill"

# Fetch the model and initialize a tokenizer
# Load model (download on first run and reference local installation for consequent runs)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Keeping track of conversation history
conversation_history = []

# Function to generate a chatbot response
def chatbot_response(user_input):
    global conversation_history

    # Create conversation history string
    history_string = "\n".join([msg for sender, msg in conversation_history if sender == "Bot"]) # Join bot messages for context

    # Tokenize the input text and history
    inputs = tokenizer.encode_plus(history_string, user_input, return_tensors="pt")

    # Generate the response from the model
    outputs = model.generate(**inputs)

    # Decode the response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    # Add interaction to conversation history as a list of [sender, message]
    conversation_history.append(["User", user_input])  # Changed to a list
    conversation_history.append(["Bot", response])    # Changed to a list

    # Update the chat history directly within the function
    return response, gr.update(value=conversation_history) # Update chat_history with conversation history


# Function to reset the conversation history
def reset_conversation():
    global conversation_history
    conversation_history = []
    # Update chat history to reflect reset
    return "Conversation reset. How can I assist you?", gr.update(value=[])  # Empty list for reset

# Create Gradio interface using with gr.Blocks() to manage the context:
with gr.Blocks() as interface:  # Create a Blocks context
    # Define input and output components
    input_box = gr.Textbox(label="Your Message", placeholder="Type your question here...")
    output_box = gr.Textbox(label="Bot Response")

    # Create a chat history display
    chat_history = gr.Chatbot(label="Chat History", show_label=False, height=400) # Changed gr.Chatbox to gr.Chatbot, removed show_line_numbers, and changed lines to height

    # Create submit button
    submit_button = gr.Button("Submit")

    # Create reset button
    reset_button = gr.Button("Reset Conversation")

    # Link the submit button to the chatbot_response function
    submit_button.click(fn=chatbot_response, inputs=input_box, outputs=[output_box, chat_history]) # outputs is now a list

    # Link the reset button to the reset function
    reset_button.click(fn=reset_conversation, outputs=[output_box, chat_history]) # outputs is now a list


# Set title and description
interface.title = "BlenderBot Chatbot"
interface.description = "A conversational AI chatbot powered by BlenderBot. Ask anything!"
interface.examples = [
    ["Hello!"],
    ["Can you tell me a joke?"],
    ["What is the capital of France?"],
]
interface.live = True

# Launch the Gradio app (for local testing)
if __name__ == "__main__":
    interface.launch()

/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:273: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://68f2bc7c3af3eebe71.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [1]:
!pip install fastapi uvicorn nest-asyncio pyngrok transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.9 MB/s eta 0:00:00


##Fourth implementation with FastAPI

In [4]:
from fastapi import FastAPI, Request
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import nest_asyncio
from pyngrok import ngrok
import uvicorn

nest_asyncio.apply()

# **Replace "YOUR_AUTHTOKEN" with your actual ngrok authtoken**
ngrok.set_auth_token("YOUR_AUTHTOKEN")

# Start ngrok tunnel
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)


# Initialize FastAPI app
app = FastAPI()

# Add CORS middleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Adjust origins as needed for production
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Model setup
model_name = "facebook/blenderbot-400M-distill"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# In-memory conversation history
conversation_history = []


# Request schema using Pydantic
class ChatRequest(BaseModel):
    prompt: str


# Root route
@app.get("/")
async def root():
    """
    Handle requests to the root path.
    """
    return {"message": "Welcome to the BlenderBot chatbot! Send your requests to /chatbot"}


@app.post("/chatbot")
async def handle_prompt(request: ChatRequest):
    """
    Handle chat requests by generating responses using the model.
    """
    input_text = request.prompt

    # Create conversation history string
    history = "\n".join(conversation_history)

    # Tokenize the input text and history
    inputs = tokenizer.encode_plus(history, input_text, return_tensors="pt")

    # Generate the response from the model
    outputs = model.generate(**inputs, max_length=60)

    # Decode the response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    # Add interaction to conversation history
    conversation_history.append(input_text)
    conversation_history.append(response)

    return {"response": response}


# Run the FastAPI app
uvicorn.run(app, port=8000)

Public URL: https://1fec-35-201-240-180.ngrok-free.app


INFO:     Started server process [2815]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
Process Process-auto_conversion:
INFO:     Shutting down
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/transformers/safetensors_conversion.py", line 84, in auto_conversion
    sha = get_conversion_pr_reference(api, pretrained_model_name_or_path, **cached_file_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/transformers/safetensors_conversion.py", line 68, in get_conversion_pr_reference
    pr = previous_pr(api, model_id, pr_title, token=token)
  File "/usr/local/lib/python3.10/dist-packages/transformers/safetensors_conversion.py", line 

After running the field above, you should see a public URL in the output(e.g., http://<ngrok_id>.ngrok.io).
- Use the public URL to access the app.
- Append /docs to view the auto-generated Swagger UI (e.g., http://<ngrok_id>.ngrok.io/docs).
- Use the /chatbot endpoint in the Swagger UI or Postman to test the chatbot.
- Alternatively use curl to query the chatbot as seen below.

In [ ]:
!curl -X POST -H "Content-Type: application/json" -d '{"prompt": "Hello, how are you?"}' https://be32-35-229-155-233.ngrok-free.app/chatbot

{"response":"Hello Hello Hello,, I am a a hell hell hell... I am good.."}

In [ ]:
!curl -X POST \
  "https://8edb-35-201-240-180.ngrok-free.app/chatbot" \
  -H "accept: application/json" \
  -H "Content-Type: application/json" \
  -d '{"prompt": "What is the capital of France?"}'


{
  "response": "I am good. I am from france. The capital of france is paris."
}

In [ ]:
!curl -X POST \
  'https://8edb-35-201-240-180.ngrok-free.app/chatbot' \
  -H 'accept: application/json' \
  -H 'Content-Type: application/json' \
  -d '{"prompt": "Can you tell me a joke?"}'